In [1]:
import csv
import openpyxl
import datetime

from io import BytesIO
import urllib

# importing pandas as pd
from bs4 import BeautifulSoup

# Pdf checker
import re
import pdfplumber

import requests

from urllib.request import urlopen

In [2]:
def total_values_pdf(date):
    # Getting url link for pdf in official page
    url = 'https://coronavirus.saude.mg.gov.br/boletim'
    u = urlopen(url)
    try:
        html = u.read().decode('utf-8')
    finally:
        u.close()

    soup = BeautifulSoup(html, "html.parser")
    url_pdf_link = soup.find('a', string=lambda value: value and value.endswith(date))["href"]

    # Request and Open PDF file
    try:
        rq = requests.get(f"https://coronavirus.saude.mg.gov.br{url_pdf_link}")
        pdf_data = pdfplumber.open(BytesIO(rq.content))
    except Exception as e:
        print(e)
    
    # Extracting data
    pattern_cases = r'^([0-9]{1,3}\.[0-9]{3}\s\.[0-9]{1,3})|^([0-9]{1,3}\.[0-9]{3}\.[0-9]{1,3})|^([0-9]{1,3}\.[0-9]{1,3})'
    pattern_deaths = r'(\b([0-9]{1,3}\.[0-9]{3}\.[0-9]{1,3})\n|\b([0-9]{1,3}\.[0-9]{1,3})\n|[0-9]{1,2}\.[0-9]\s\s[0-9]{1,3}\s\n)'
    text = pdf_data.pages[0].extract_text()
    total_confirmed = re.search(pattern_cases, text, re.MULTILINE)[0]
    total_death = re.search(pattern_deaths, text, re.MULTILINE)[0]
    # removing dots and breaklines
    total_confirmed = re.sub(r"\.|\n|\s", "", total_confirmed)
    total_death = re.sub(r"\.|\n|\s", "", total_death)
    return [int(total_confirmed), int(total_death)]

In [3]:
# Loading excel file from url
url = "https://coronavirus.saude.mg.gov.br/images/microdados/xlsx_painel.xlsx"
file = urllib.request.urlopen(url).read()
wb = openpyxl.load_workbook(filename = BytesIO(file), read_only=True)
pages = wb.sheetnames

In [4]:
# Getting cities IBGE code
with open("cod_mun_MG.csv", mode="r", encoding="utf-8") as fobj:
    reader = csv.reader(fobj)
    mun_cod_dict = {str(row[1])[:6]:row[0] for row in reader}

In [5]:
# Excel file basedo on page returning dict city_code, value and date
def excel_data(page):
    ws = wb[pages[page]]
    data = [list(row) for row in ws.iter_rows(values_only=True)]

    headers = [data[0][3], data[0][1], data[0][2]] 
    values = [[row[3], row[1], row[2]] for row in data[1:]]
    
    result = []
    for value in values:
        result.append(dict(zip(headers, value)))
    return result

In [6]:
# Dates to generate csv
dates = ["2021-04-23"]
# Loop to generate based on date
for date in dates:
    # Getting total values from pdf
    total_confirmed_pdf, total_death_pdf = total_values_pdf(
        "/".join(date.split("-")[::-1]))

    date = datetime.datetime.strptime(date, '%Y-%m-%d').date()
    # Scraping data and generating csv
    confirmed = excel_data(0)
    death = excel_data(1)
    result = []
    for key, val in mun_cod_dict.items():
        key = str(key)
        result.append([val,
                       sum([item["NUM_CASOS"] for item in confirmed if str(
                           item["CodigoIBGE"]) == key and item["DATA"].date() <= date]),
                       sum([item["NUM_OBITOS"] for item in death if str(
                           item["CodigoIBGE"]) == key and item["DATA"].date() <= date])
                       ])
    # Generating csv
    output_file = f"MG_{date}.csv"
    with open(output_file, mode="w", encoding="utf-8", newline="") as fobj:
        writer = csv.DictWriter(
            fobj, fieldnames=["municipio", "confirmados", "mortes"])
        writer.writeheader()

        confirmed_total = 0
        death_total = 0
        mun_result = []
        for municipio in result:
            confirmed_total += int(municipio[1])
            death_total += int(municipio[2])
            mun_result.append(
                {
                    "municipio": municipio[0],
                    "confirmados": municipio[1],
                    "mortes": municipio[2],
                }
            )
        # Importados e Indeninidos
        imp_ind_confirmed = sum(
            [item["NUM_CASOS"] for item in confirmed if item["CodigoIBGE"] == None and item["DATA"].date() <= date])
        imp_ind_deaths = sum(
            [item["NUM_OBITOS"] for item in death if item["CodigoIBGE"] == None and item["DATA"].date() <= date])

        if total_confirmed_pdf == confirmed_total + imp_ind_confirmed:
            confirmed_total = total_confirmed_pdf
        else:
            print("Fail", output_file,
                  "Comparação valores de confirmados apresentou resultados diferentes")
            confirmed_total = 0
        if total_death_pdf == death_total + imp_ind_deaths:
            death_total = total_death_pdf
        else:
            print("Fail", output_file,
                  "Comparação valores de mortes apresentou resultados diferentes")
            death_total = 0
        writer.writerows(mun_result)
        writer.writerow(
            {
                "municipio": "Importados/Indefinidos",
                "confirmados": imp_ind_confirmed,
                "mortes": imp_ind_deaths
            }
        )
        writer.writerow(
            {
                "municipio": "TOTAL NO ESTADO",
                "confirmados": confirmed_total,
                "mortes": death_total,
            }
        )
